In [3]:
!conda env list

# conda environments:
#
VAdepthENV               /home/013907062/.conda/envs/VAdepthENV
cmpe249                  /home/013907062/.conda/envs/cmpe249
env_onmttf               /home/013907062/.conda/envs/env_onmttf
hawaii                   /home/013907062/.conda/envs/hawaii
hawaii_hf             *  /home/013907062/.conda/envs/hawaii_hf
koen_base                /home/013907062/.conda/envs/koen_base
newDepth                 /home/013907062/.conda/envs/newDepth
seamless                 /home/013907062/.conda/envs/seamless
test                     /home/013907062/.conda/envs/test
wmt_infer                /home/013907062/.conda/envs/wmt_infer
base                     /opt/ohpc/pub/apps/anaconda/3.9
stylegan2                /opt/ohpc/pub/apps/anaconda/3.9/envs/stylegan2



In [4]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 26.8 MB/s eta 0:00:00 0:00:01


## TEST for English Tagalog

In [47]:
from transformers import pipeline
from transformers import AutoTokenizer

#https://huggingface.co/Helsinki-NLP
'''
Pipeline supports running on CPU or GPU through the device argument.
Users can specify device argument as an integer, -1 meaning “CPU”, >= 0 referring the CUDA device ordinal.
'''
model_checkpoint = "Helsinki-NLP/opus-mt-en-tl"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

[{'translation_text': 'Paglubog sa pinalawak na mga sinulid'}]

In [48]:
with open('jmbt22_tatoeba-en-tgl.en') as f:
    en_sentence = f.readlines()

with open('jmbt22_tatoeba-en-tgl.tl') as f:
    tl_sentence = f.readlines()

In [30]:
from tqdm import tqdm
import time

In [33]:
# English to Tagalog

results_eng2tgl = []

for idx, line in enumerate(tqdm(en_sentence)): 
    translated_text = translator(line)
    results_eng2tgl.append(translated_text)

with open("Helsinki-NLP_jmbt22_tatoeba-en-tgl.pred.eng2tgl", "w") as file:
    for row in results_eng2tgl:
        if len(row) != 0:
            file.write(str(row) + '\n')


100%|██████████| 7300/7300 [20:52<00:00,  5.83it/s]


In [50]:
with open('Helsinki-NLP_jmbt22_tatoeba-en-tgl.pred.eng2tgl') as f:
    en_preds = f.readlines()

print(len(en_preds))

with open("Helsinki-NLP_jmbt22_tatoeba-en-tgl.pred.eng2tgl_refine", "w") as file:
    for pred in en_preds:
        file.write(pred[23:-4]+'\n')

7300


In [51]:
!sacrebleu jmbt22_tatoeba-en-tgl.tl < Helsinki-NLP_jmbt22_tatoeba-en-tgl.pred.eng2tgl_refine

{
 "name": "BLEU",
 "score": 26.4,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "56.7/31.8/20.5/13.2 (BP = 1.000 ratio = 1.091 hyp_len = 56810 ref_len = 52063)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


In [52]:
# Tagalog To English download

model_checkpoint = "Helsinki-NLP/opus-mt-tl-en"

tokenizerTlEn = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")
translatorTlEn = pipeline("translation", model=model_checkpoint)
translatorTlEn("Paglubog sa pinalawak na mga sinulid")

[{'translation_text': 'Dipping out the expanded threads'}]

In [55]:
# Tagalog To English inference
results_tgl2eng = []

for idx, line in enumerate(tqdm(tl_sentence)): 
    translated_text = translatorTlEn(line)
    results_tgl2eng.append(translated_text[0]['translation_text'])


with open("Helsinki-NLP_jmbt22_tatoeba-en-tgl.pred.tgl2eng", "w") as file:
    for row in results_tgl2eng:
        if len(row) != 0:
            file.write(str(row) + '\n')


100%|██████████| 7300/7300 [25:29<00:00,  4.77it/s]


In [56]:
!sacrebleu jmbt22_tatoeba-en-tgl.en < Helsinki-NLP_jmbt22_tatoeba-en-tgl.pred.tgl2eng

{
 "name": "BLEU",
 "score": 35.1,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "66.0/41.5/29.2/20.9 (BP = 0.976 ratio = 0.977 hyp_len = 51848 ref_len = 53094)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


### dataset flores200_devtest

In [57]:
file_eng = "dataset_evaluation/flores200_dataset/devtest/eng_Latn.devtest"
file_tgl = "dataset_evaluation/flores200_dataset/devtest/tgl_Latn.devtest"

with open(file_eng) as f:
    en_sentence = f.readlines()

with open(file_tgl) as f:
    tl_sentence = f.readlines()

print(len(en_sentence))
print(len(tl_sentence))

1012
1012


In [63]:
# English to Tagalog

results_eng2tgl = []

for idx, line in enumerate(tqdm(en_sentence)): 
    translated_text = translator(line)
    results_eng2tgl.append(translated_text[0]['translation_text'])

with open("Helsinki-NLP_flores200_dataset-en-tgl.pred.eng2tgl", "w") as file:
    for row in results_eng2tgl:
        file.write(str(row) + '\n')

100%|██████████| 1012/1012 [14:06<00:00,  1.19it/s]


sacreBLEU: System and reference streams have different lengths.


In [65]:
#file_eng = "dataset_evaluation/flores200_dataset/devtest/eng_Latn.devtest"
#file_tgl = "dataset_evaluation/flores200_dataset/devtest/tgl_Latn.devtest"

!sacrebleu dataset_evaluation/flores200_dataset/devtest/tgl_Latn.devtest < Helsinki-NLP_flores200_dataset-en-tgl.pred.eng2tgl

{
 "name": "BLEU",
 "score": 34.5,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "67.5/41.8/28.1/19.5 (BP = 0.978 ratio = 0.978 hyp_len = 28961 ref_len = 29603)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


In [69]:
# Tagalog To English inference
results_tgl2eng = []

for idx, line in enumerate(tqdm(tl_sentence)): 
    translated_text = translatorTlEn(line)
    results_tgl2eng.append(translated_text[0]['translation_text'])

with open("Helsinki-NLP_flores200_dataset-en-tgl.pred.tgl2eng", "w") as file:
    for row in results_tgl2eng:
        file.write(str(row) + '\n')

!sacrebleu dataset_evaluation/flores200_dataset/devtest/eng_Latn.devtest < Helsinki-NLP_flores200_dataset-en-tgl.pred.tgl2eng

100%|██████████| 1012/1012 [16:00<00:00,  1.05it/s]


{
 "name": "BLEU",
 "score": 34.5,
 "signature": "nrefs:1|case:mixed|eff:no|tok:13a|smooth:exp|version:2.3.1",
 "verbose_score": "65.4/41.0/27.7/19.1 (BP = 1.000 ratio = 1.023 hyp_len = 25285 ref_len = 24721)",
 "nrefs": "1",
 "case": "mixed",
 "eff": "no",
 "tok": "13a",
 "smooth": "exp",
 "version": "2.3.1"
}


## Test set preprocessing

In [18]:
with open('jmbt22_tatoeba-en-tgl.en') as f:
    en_sentence = f.readlines()

with open('jmbt22_tatoeba-en-tgl.tl') as f:
    tl_sentence = f.readlines()

In [24]:
tokenized_sentences = tokenizer(
        en_sentence, text_target=tl_sentence, max_length=512, truncation=True
    )

In [25]:
print((tokenized_sentences["input_ids"][1]))
print((tokenized_sentences["attention_mask"][1]))
print((tokenized_sentences["labels"][1]))

[33, 51, 12, 429, 12, 2457, 3, 0]
[1, 1, 1, 1, 1, 1, 1, 1]
[1160, 80, 56, 8302, 3, 0]


In [27]:
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

d = {'test':Dataset.from_dict({'input_ids':tokenized_sentences["input_ids"],
                               'attention_mask':tokenized_sentences["attention_mask"],
                               'labels':tokenized_sentences["labels"]})
    }
eval_dataset_dic = DatasetDict(d)
eval_dataset_dic

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 7300
    })
})

In [ ]:
import numpy as np
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
from torch.utils.data import DataLoader
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

eval_dataset_dic.set_format("torch")
eval_dataloader = DataLoader(
    eval_dataset_dic["test"], collate_fn=data_collator, batch_size=8
)
